In [42]:
! nvidia-smi

Mon May 30 05:16:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [43]:
!pip install pytorch-tabnet optuna pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score,classification_report
import optuna as opt
import torch
import os
import joblib

In [45]:
def make_save_cv_model(i,model_name,model,best_params,optim,output_path="./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/cross_validated_models"):

    ''' This function saves cross validation model in the corresponding directory ( if the path does not exist it creates the path for it'''


    if os.path.exists(os.path.join(output_path,f"{i}_{model_name}_{optim}")):
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))
    else:
        os.mkdir(os.path.join(output_path,f"{i}_{model_name}_{optim}"))
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))

In [46]:
def train(fold_dict,fold,model_name,sc_df,tar_col,optim,optim_trial,k_folds=10,tar_cols="",verbose=1):

    ''' this function is used to train the model with parameters optimization using optuna and cross validation using stratified k_folds'''

    y = sc_df[tar_col]
    x = sc_df.drop([tar_col],axis=1)
    model_name = model_name 
    def objective(trial):
      train_index = fold_dict[fold]["train"]
      test_index = fold_dict[fold]["test"]
      clf = TabNetClassifier(n_d=trial.suggest_int("n_d", 8, 64),
                              n_a =trial.suggest_int("n_a", 8, 64),
                              n_steps = trial.suggest_int("n_steps",3,10),
                              gamma =trial.suggest_float("gamma", 1.0, 2.0),
                              n_independent = trial.suggest_int("n_independent",1,5),
                              n_shared = trial.suggest_int("n_shared",1,5),
                              momentum = trial.suggest_float("momentum", 0.01, 0.4),
                              optimizer_fn = torch.optim.Adam,
                              # scheduler_fn = torch.optim.lr_scheduler,
                              # scheduler_params = {"gamma" :trial.suggest_float("sch-gamma", 0.5, 0.95), "step_size": trial.suggest_int("sch_step_size", 10, 20, 2)},
                              verbose = verbose,
                              device_name = "auto"
                              )
      # print(f" train_index :: {train_index}")
      # print(f" test_index :: {test_index}")
      X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
      # print(X_train.shape, X_test.shape)
      X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
      Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
      Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
      print(Y_train.shape, Y_test.shape)
      clf.fit(X_train, Y_train,
              eval_set=[(X_test, Y_test)],
              eval_metric=['accuracy'])
      Y_pred = clf.predict(X_test)
      print(classification_report(Y_test, Y_pred, labels=[x for x in range(6)]))
      acc = accuracy_score(Y_pred, Y_test)
      return acc

    print(f"Starting optimization for fold : [{fold}/{k_folds}]")
    study = opt.create_study(direction='maximize')
    study.optimize(objective, n_trials=optim_trial)
    best_params = study.best_params
    print(f" Best params for fold : [{fold}/{k_folds}]")
    print(best_params)
    joblib.dump(best_params,f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/comp/fold_{fold}_best_params.z")
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/fold_{fold}_best_params.txt", "w+") as file:file.write(str(best_params))
    print(f"Saved best_params at : outputs/{model_name}/best_params/fold_{fold}_best_params.txt")
    train_index = fold_dict[fold]["train"]
    test_index = fold_dict[fold]["test"]
    X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    # print(X_train.shape, X_test.shape)
    X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
    clf_model = TabNetClassifier(**study.best_params)
    clf_model.fit(X_train,Y_train)
    Y_pred = clf_model.predict(X_test)
    clf_report = classification_report(Y_test, Y_pred, labels=[x for x in range(6)])
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/classification_report/{model_name}_{fold}_classification_report.txt","w+") as file:file.write(str(clf_report))
    accuracy = accuracy_score(Y_pred, Y_test)
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/{model_name}_{fold}_accuracy_score.txt","w+") as file:file.write(f" accuracy :: {str(accuracy)}")
    try:
        print("[++] Saving the model and parameters in corresponding directories")
        make_save_cv_model(fold,model_name,clf_model,best_params,optim=optim)
    except:
        print("[-] Failed to save the model")

In [47]:
use_df = pd.read_csv("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/data/trainable_scaled_balanced.csv")
tar_col = "PCE_categorical"
model_name = "pytorch_tabnet"
optimizer = "Adam"
fold_dict = joblib.load("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/inputs/fold_vals/fold_data.z")
fold = 0

In [48]:
train(fold_dict = fold_dict,
      fold = fold,
      model_name=model_name,
      sc_df=use_df,
      tar_col=tar_col,
      optim=optimizer,
      optim_trial = 15)
print(f"[++] Ended the training process for fold {fold}")

[I 2022-05-30 05:17:07,108] A new study created in memory with name: no-name-88022efc-35f7-4976-9b0c-0efae2567077


Starting optimization for fold : [0/10]
Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.50203 | val_0_accuracy: 0.54361 |  0:00:02s
epoch 1  | loss: 1.00203 | val_0_accuracy: 0.60125 |  0:00:04s
epoch 2  | loss: 0.94154 | val_0_accuracy: 0.59089 |  0:00:06s
epoch 3  | loss: 0.85814 | val_0_accuracy: 0.62824 |  0:00:08s
epoch 4  | loss: 0.82792 | val_0_accuracy: 0.61701 |  0:00:10s
epoch 5  | loss: 0.80023 | val_0_accuracy: 0.64616 |  0:00:12s
epoch 6  | loss: 0.78621 | val_0_accuracy: 0.63169 |  0:00:14s
epoch 7  | loss: 0.7713  | val_0_accuracy: 0.65134 |  0:00:16s
epoch 8  | loss: 0.7436  | val_0_accuracy: 0.66343 |  0:00:19s
epoch 9  | loss: 0.73004 | val_0_accuracy: 0.68092 |  0:00:21s
epoch 10 | loss: 0.71968 | val_0_accuracy: 0.67638 |  0:00:23s
epoch 11 | loss: 0.72954 | val_0_accuracy: 0.67379 |  0:00:25s
epoch 12 | loss: 0.72698 | val_0_accuracy: 0.66559 |  0:00:27s
epoch 13 | loss: 0.71469 | val_0_accuracy: 0.66753 |  0:00:29s
epoch 14 | loss: 0.6992  | val_0_accuracy:

[I 2022-05-30 05:20:51,884] Trial 0 finished with value: 0.7789291882556131 and parameters: {'n_d': 27, 'n_a': 16, 'n_steps': 5, 'gamma': 1.8641624835897033, 'n_independent': 2, 'n_shared': 1, 'momentum': 0.37639041518072286}. Best is trial 0 with value: 0.7789291882556131.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       825
           1       0.63      0.55      0.59       826
           2       0.78      0.92      0.85       822
           3       1.00      1.00      1.00       779
           4       0.52      0.59      0.55       761
           5       0.72      0.56      0.63       619

    accuracy                           0.78      4632
   macro avg       0.78      0.77      0.77      4632
weighted avg       0.78      0.78      0.78      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.48554 | val_0_accuracy: 0.53735 |  0:00:03s
epoch 1  | loss: 0.97484 | val_0_accuracy: 0.61917 |  0:00:07s
epoch 2  | loss: 0.87816 | val_0_accuracy: 0.60276 |  0:00:10s
epoch 3  | loss: 0.81769 | val_0_accuracy: 0.63148 |  0:00:14s
epoch 4  | loss: 0.7671  | val_0_accuracy: 0.65199 |  0:00:17s
epoch 5  | loss: 0.73441 | val_0_accuracy: 0.6494  |  0:00:20s
epoch 6  | loss: 0.72308 | val_0_accuracy: 

[I 2022-05-30 05:24:53,127] Trial 1 finished with value: 0.7793609671848014 and parameters: {'n_d': 15, 'n_a': 60, 'n_steps': 4, 'gamma': 1.3766835406477922, 'n_independent': 3, 'n_shared': 5, 'momentum': 0.17289579110815514}. Best is trial 1 with value: 0.7793609671848014.


...

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       825
           1       0.68      0.61      0.64       826
           2       0.83      0.92      0.87       822
           3       1.00      1.00      1.00       779
           4       0.53      0.63      0.58       761
           5       0.73      0.57      0.64       619

    accuracy                           0.80      4632
   macro avg       0.79      0.79      0.79      4632
weighted avg       0.80      0.80      0.80      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.10471 | val_0_accuracy: 0.62737 |  0:00:01s
epoch 1  | loss: 0.81423 | val_0_accuracy: 0.65889 |  0:00:03s
epoch 2  | loss: 0.75593 | val_0_accuracy: 0.66731 |  0:00:04s
epoch 3  | loss: 0.7238  | val_0_accuracy: 0.68459 |  0:00:06s
epoch 4  | loss: 0.70151 | val_0_accuracy: 0.69344 |  0:00:07s
epoch 5  | loss: 0.69142 | val_0_accuracy: 0.69473 |  0:00:09s
epoch 6  | loss: 0.67903 | val_0_accuracy: 

[I 2022-05-30 06:25:04,722] Trial 13 finished with value: 0.7914507772020726 and parameters: {'n_d': 44, 'n_a': 47, 'n_steps': 3, 'gamma': 1.7655488605916851, 'n_independent': 1, 'n_shared': 2, 'momentum': 0.22725916640028}. Best is trial 12 with value: 0.7968480138169257.


              precision    recall  f1-score   support

           0       0.99      1.00      1.00       825
           1       0.64      0.57      0.60       826
           2       0.77      0.95      0.85       822
           3       1.00      1.00      1.00       779
           4       0.57      0.56      0.56       761
           5       0.73      0.62      0.67       619

    accuracy                           0.79      4632
   macro avg       0.78      0.78      0.78      4632
weighted avg       0.79      0.79      0.79      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.68401 | val_0_accuracy: 0.49374 |  0:00:04s
epoch 1  | loss: 1.24467 | val_0_accuracy: 0.49784 |  0:00:09s
epoch 2  | loss: 1.17703 | val_0_accuracy: 0.54555 |  0:00:13s
epoch 3  | loss: 1.04643 | val_0_accuracy: 0.60212 |  0:00:18s
epoch 4  | loss: 0.85894 | val_0_accuracy: 0.61636 |  0:00:22s
epoch 5  | loss: 0.82405 | val_0_accuracy: 0.63623 |  0:00:27s
epoch 6  | loss: 0.78119 | val_0_accuracy: 

[I 2022-05-30 06:32:41,423] Trial 14 finished with value: 0.7994386873920553 and parameters: {'n_d': 42, 'n_a': 53, 'n_steps': 6, 'gamma': 1.9837494877637831, 'n_independent': 5, 'n_shared': 2, 'momentum': 0.3157789464830161}. Best is trial 14 with value: 0.7994386873920553.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       825
           1       0.68      0.61      0.65       826
           2       0.83      0.95      0.88       822
           3       1.00      1.00      1.00       779
           4       0.55      0.56      0.55       761
           5       0.69      0.63      0.66       619

    accuracy                           0.80      4632
   macro avg       0.79      0.79      0.79      4632
weighted avg       0.80      0.80      0.80      4632

 Best params for fold : [0/10]
{'n_d': 42, 'n_a': 53, 'n_steps': 6, 'gamma': 1.9837494877637831, 'n_independent': 5, 'n_shared': 2, 'momentum': 0.3157789464830161}
Saved best_params at : outputs/pytorch_tabnet/best_params/fold_0_best_params.txt
Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 1.68401 |  0:00:04s
epoch 1  | loss: 1.28486 |  0:00:08s
epoch 2  | loss: 1.11598 |  0:00:12s
epoch 3  

Fold 0 has started running on 20-05-22 


Fold 0 has completed sucessfully on 17:00 20-05-22

Fold 1 has started running at 15:15 21-05-22

Fold 2 has started running at 09:45 22-05-22

Fold 2 has completed sucessfully on 10:58 22-05-22

Fold 3 has started running at 18:40 22-05-22

Fold 3 has completed sucessfully on 22-05-22

Fold 4 completed sucessfully on 21:04 on 22-05-22

Fold 5 started at 18:21 on 23-05-22

Fold 5 completed sucessfully on 19:44 on 23-05-22

Fold 6 started at 12:53 on 24-05-22

Fold 6 has completed at 14:14 on 24-05-22

Fold 7 started at 14:18 on 24-05-22

Fold 7 execution failed due to colab gpu time limit

Fold 7 trial 1 started at 11:00 on 25-05-22

Fold 7 has completed sucessfully at 12:14 on 25-05-22 

Fold 8 has started at 9:38 on 26-05-22

Fold 8 filed due to interrupted internet connection

Fold 8 trial 1 started at 13:38 on 26-05-22

Fold 8 has successfully executed at 15:33 on 26-05-22

Fold 9 has started at 13:35 on 27-05-22

Fold 9 has completed at 14:55 on 27-05-22

Editing with rectified dataset witout duplicacy because of space values

Fold 0 started at 13:21 on 28-05-22

Fold 0 completed sucessfully at 14:46 on 28-05-22

Fold 1 Failed to run due to some index error

_____ Restarting the training process again due to data distribution failure____



Fold 0 started at 10:47 on 30-05-22

Fold 0 completed successfully at 12:30 on 30-05-22